In [1]:
!pip install --upgrade category_encoders matplotlib pandas_profiling seaborn psutil xlrd
import sys, getpass, pandas_profiling
a = getpass.getuser()
### examples appending of folder
# A
sys.path.append('/home/{}/Code/Github-Repos/BFDS_Library/'.format(a))
# B
#sys.path.append('/Users/{}/Programs/BFDS_Library/'.format(a))

# Pull in library if in Google Colab
if 'google.colab' in sys.modules:
    os.chdir('/content')
    !git init .
    !git remote add origin https://github.com/jeibloo/BFDS_Library.git
    !git pull origin master

#from BFDS_Library import ACompass, BWarp, CToolbox 
import AMap as am
import BCook as bc
import CTools as ct

Requirement already up-to-date: category_encoders in /home/ygeun/.local/share/virtualenvs/coral-coral-coral-qINAJzRq/lib/python3.7/site-packages (2.0.0)
Requirement already up-to-date: matplotlib in /home/ygeun/.local/share/virtualenvs/coral-coral-coral-qINAJzRq/lib/python3.7/site-packages (3.1.1)
Requirement already up-to-date: pandas_profiling in /home/ygeun/.local/share/virtualenvs/coral-coral-coral-qINAJzRq/lib/python3.7/site-packages (2.3.0)
Requirement already up-to-date: seaborn in /home/ygeun/.local/share/virtualenvs/coral-coral-coral-qINAJzRq/lib/python3.7/site-packages (0.9.0)
Requirement already up-to-date: psutil in /home/ygeun/.local/share/virtualenvs/coral-coral-coral-qINAJzRq/lib/python3.7/site-packages (5.6.3)
Requirement already up-to-date: xlrd in /home/ygeun/.local/share/virtualenvs/coral-coral-coral-qINAJzRq/lib/python3.7/site-packages (1.2.0)


Cool dataset from [Reefbase](http://www.reefbase.org/gis_maps/useofmaps.aspx).

In [2]:
import pandas as pd
pd.options.display.float_format = None
df = pd.read_excel('./dataset/CoralBleaching.xlsm')

In [3]:
target = 'BLEACHING_SEVERITY'

In [4]:
def wgle(df):
    # Group by bleaching or not
    df[target] = df[target].replace({
        'Low':1, 'HIGH':1, 'Medium':1, 'Severity Unknown':1,
        'No Bleaching':0})

    # Get all features for ease-of-use
    features, num_feats, cat_feats = bc.pantryFeatures(df)
    
    # Drop columns with almost no values.
    df = bc.diceUglyCols(df, features, percent=50)
    
    # Special weird values
    df['LAT'] = df['LAT'].replace({-10269:-10.269})
    
    return df

In [5]:
df.profile_report()

In [6]:
df_wrangle = wgle(df)

In [7]:
print("Cut down {} columns.".format(df.shape[1]-df_wrangle.shape[1]))

Cut down 15 columns.


In [8]:
# Split into Train Test & Val by year
test_mask = (df['YEAR'] > 1999)
vali_mask = ((df['YEAR'] > 1996) & (df['YEAR'] <= 1999))
trin_mask = (df['YEAR'] <= 1995)
# Use masks
test = df[test_mask].copy()
trin = df[trin_mask].copy()
vali = df[vali_mask].copy()

In [9]:
print("ASSURANCE CHECK\nTest shape:{}\nTrain shape:{}\nValidation shape:{}".format(
    test.shape,trin.shape,vali.shape)
)

ASSURANCE CHECK
Test shape:(3487, 28)
Train shape:(407, 28)
Validation shape:(2275, 28)


In [10]:
### Chop the sets up
features, _, _ = bc.pantryFeatures(trin)
X_train = trin[features].drop(target,axis=1)
y_train = trin[target]

X_test = test[features].drop(target,axis=1)
y_test = test[target]

X_vali = vali[features].drop(target,axis=1)
y_vali = vali[target]

print("ASSURANCE CHECK\nTrain cols:\t{}\nValidtn cols:\t{}\nTest cols:\t{}".format(
    len(X_train.columns)+1,
    len(X_test.columns)+1,
    len(X_vali.columns)+1)
)

ASSURANCE CHECK
Train cols:	28
Validtn cols:	28
Test cols:	28


In [11]:
# Majority Class Baseline
bc.soupBaseModel(df, target, 1)

Classification
1    0.761389
0    0.238611
Name: BLEACHING_SEVERITY, dtype: float64


In [12]:
### Start fitting this classification model
import category_encoders as ce
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline

In [13]:
pipey = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(),
    RandomForestClassifier(
        n_estimators=500,
        class_weight='balanced',
        n_jobs=-1,
        random_state=4
    )
)

In [19]:
from sklearn.metrics import roc_auc_score
# Train the model on these two beauts
pipey.fit(X_train, y_train);

In [20]:
# Now predict what y_val might be
y_pred_proba = pipey.predict_proba(X_vali)[:,1]

In [28]:
y_vali.dtype, y_pred_proba.dtype

(dtype('int64'), dtype('float64'))

In [29]:
am.plotConfusionMatrix?

In [ ]:
# Print out what y_val prediction was
print("Val Set AUC ROC: {}".format(roc_auc_score(y_vali,y_pred_proba)))

In [ ]:
# Test cross-validation
from sklearn.model_selection import cross_val_score
cross_val_score(pipey, X_train, y_train, cv=3, scoring='roc_auc')

Why is it perfect...

In [ ]:
X_combo = pd.concat([X_train, X_vali])
y_combo = pd.concat([y_train, y_vali])
# Train the pipeline on the combo
pipey.fit(X_combo, y_combo)
# Predict probablity on final test set X's
y_pred_proba = pipey.predict_proba(X_test)[:,1]
# Now compare against y_test values
print("Test Set AUC ROC: {}".format(roc_auc_score(y_test,y_pred_proba)))